In [1]:
import airsim
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import splprep, splev

from utils import Gate 
from client import Simulator


sim = Simulator()
sim.get_gates(7)
# get gate_2 manually because there is some bug in the api
sim.gates[1] = Gate.from_unreal_position(-300, 7640, 100)
p = sim.track_position_matrix()

# interpolation (3rd degree with some smoothing)
tck, u = splprep(p.T, u=None, s=5.0, per=1, k=3) 
u_new = np.linspace(u.min(), u.max(), 1000)
ev = np.array(splev(u_new, tck, der=0)).T

# plot
def plot_gates(xx, yy):
    fig, ax = plt.subplots()
    plt.plot(ev[:,xx], ev[:,yy], 'b--')
    ax.scatter(p[:,xx], p[:,yy])
    for i in range(7):
        ax.annotate(str(i+1), (p[i, xx], p[i, yy]))

Exception: Unreal Engine simulation is not running